# Virtual staining by image translation
***

This notebook illustrates image translation with deep convolutional neural networks (CNN). We frame the image translation as a regression problem solved using a residual U-Net model. The notebook demonstrates how to translate quantitative phase images of mouse kidney tissue to the fluorescence images of nuclei using data and model reported in our paper(https://doi.org/10.7554/eLife.55502).
<div>
    <img src="virtualstaining_summary.png"  width="950">
</div>

##  Overview <a class="anchor" id="overview"></a>

U-Net model consists of an encoder (downsampling) part and a decoder (upsampling) part. The U-Net model is immensely popular for many image analysis tasks. One of the key design feature of the U-Net architecture is the skip connections between the encoder and decoder layers, which allows the model to learn patterns at multiple spatial resolutions. U-Nets were orginally designed for image segmentation (https://link.springer.com/chapter/10.1007/978-3-319-24574-4_28). Using U-Nets for image translation (a regression problem) needs a few tweaks, such as: 
* changing the loss function, 
* changing the final activation layer, and 
* data normalization (more on that later).

Below is an overview of the 2D and 2.5D residual U-Net model architecture. The main differences from the original U-Net architecture are: 
1. We add short skip connection within each block (residual block) in addition to the long skip connections in the original U-Net. 
2. The long skip connections in the 2.5D model project the data along depth dimension - it uses short stacks (5-7 slices) as input to compute a 2D output. This architecture provides better translation accuracy than 2D model, because the model learns the pattern of blur along depth. 
3. The 2.5D model can be used to virtually stain 3D stacks by sliding the input window over the data along $x$, $y$, and $z$ dimensions.

<div>
    <img src="supp_modelarch_RGB.png"  width="950">
</div>
The 2D model translates slice$\rightarrow$slice, whereas 2.5D model translates stack$\rightarrow$slice. 

We'll be using the architecture similar to above, but with same convolution instead of valid convolution.  Same convolution operation pads the input image so that the output image has the same size as the input image after convolution.

***
The first half of the exercise will focus on predicting nuclei from label-free measurements using different flavors of 2D UNets in [microDL](https://github.com/mehta-lab/microDL) in the jupyter notebook. 
In the second half, you can fine tune the model or explore translation between different types of modalities. If you are comfortable using CLI, you can potentially train 2-3 small models in parallel on your GPU using config files.
***

## Set path and import libraries

In [ ]:
import os
import sys

TOP_DIR = os.path.expanduser('.') # Top directory in which microDL library, data, patches, and trained models are saved.'.' means current directory. 
INPUT_DIR = os.path.join(TOP_DIR, 'data') # directory where raw data is saved.
TILE_DIR = os.path.join(TOP_DIR, 'phase2dna_microdl_patches') # directory where patches will be saved.
MODEL_DIR = os.path.join(TOP_DIR, 'microdl_model') # directory where model will be saved.
module_path = os.path.join(TOP_DIR, 'microDL') #directory of microDL module.

# make sure that microDL library can be seen by this interpreter.
if module_path not in sys.path:
    sys.path.append(module_path)
print(sys.path)

In [ ]:
# autoreload modules if they have changed.
%load_ext autoreload
%autoreload 2

import glob
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import importlib
import pandas as pd
import pprint

# Supress warnings related to deprecation of tensorflow 1.x    
import warnings
warnings.filterwarnings("ignore", message=r"Passing", category=FutureWarning)
warnings.filterwarnings("ignore", message=r"DEPRECATED", category=UserWarning)

import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)

import micro_dl.utils.meta_utils as meta_utils
import micro_dl.cli.train_script as train
import micro_dl.cli.preprocess_script as preprocess
import micro_dl.utils.aux_utils as aux_utils
import micro_dl.utils.image_utils as im_utils
import micro_dl.utils.masks as mask_utils
import micro_dl.utils.normalize as norm_utils
import micro_dl.utils.train_utils as train_utils
import micro_dl.inference.image_inference as image_inf
import micro_dl.cli.metrics_script as metrics
    
# Setup pretty printing
pp = pprint.PrettyPrinter(indent=4)


font = {'family' : 'DejaVu Sans',
        'size'   : 20}

matplotlib.rc('font', **font)
%matplotlib inline


## 1. Explore data and metadata

Data examination and curation is a *very* important part of training accurate and useful models. ML researchers typically spend half or more of their time curating the right dataset for the problem at hand. In this case, we will start with a small dataset to quickly iterate on the parameters of a virtual staining model, before refining the accuracy with microDL.

### Explore input and target images

The goal of this exercise is to translate 2D slices of phase or retardance image into 2D slices of fluorescene images of F-actin and DNA. The specimen is a mouse kidney tissue section. Let's start by looking at some example images.

<div class="alert alert-block alert-info"><h4>
    TODO 1: Explore the label-free and fluorescence images of the same FOV.

In [17]:
# hint: paths look like this.
position = 5
actin_path = os.path.join(INPUT_DIR, f'img_568_t000_p{position:03}_z010.tif') #F-actin was imaged with Alexa Fluor 568 using 468nm excitation wavelength.
dna_path=os.path.join(INPUT_DIR, f'img_405_t000_p{position:03}_z010.tif') # DNA was imaged with Hoechst using 405nm excitation wavelength.
phase_path=os.path.join(INPUT_DIR,f'img_phase_t000_p{position:03}_z010.tif') # the phase and polarization were imaged using 530nm wavelength.
ret_path=os.path.join(INPUT_DIR,f'img_Retardance_t000_p{position:03}_z010.tif') # the phase and polarization were imaged using 530nm wavelength.


### Metadata <a class="anchor" id="metadata"></a>

microDL uses CSV file to track the metadata of the images. We parse metadata data using a widely used [pandas](https://pandas.pydata.org/) library.  We'll generate the metadata and take a look:

In [ ]:
meta_utils.frames_meta_generator(INPUT_DIR)
meta_utils.ints_meta_generator(INPUT_DIR,
                               num_workers=8,
                               )

In [ ]:
frames_meta = pd.read_csv(os.path.join(INPUT_DIR, 'frames_meta.csv'), index_col=0)
frames_meta

Each channel in our data is assgined a unique channel ID. We'll reference the channels by their IDs in the config files. Let's check the channel IDs. `pandas` frames can be indexed into like numpy arrays where index can be the headers of a column. See more examples [here](https://pandas.pydata.org/docs/user_guide/indexing.html).


In [ ]:
chan_id_df = frames_meta[['channel_idx', 'channel_name']].drop_duplicates()
chan_id_df

In [ ]:
# Find position ids 
pos_id_df = frames_meta[['pos_idx']].drop_duplicates()
pos_id_df

### Checkpoint 1

<div class="alert alert-block alert-success"><h4>
    
Let us know by slack if you have been able to load the data and generate metadata table.

## 2. Preprocessing <a class="anchor" id="preproc"></a>

We will  be using three main modules of microDL for image translation: preprocessing, training and inference (see imports above).

The first step is the preprocessing. The key step in the preprocessing is tiling the images. The original image size (2048 x 2048 pixels) is too large to be able to fit into memory and similar structures reappear across the images. So we'd be much better off splitting the images into smalle pieces (patches). A design consideration for convolution neural networks is the [receptive field](https://medium.com/mlreview/a-guide-to-receptive-field-arithmetic-for-convolutional-neural-networks-e0f514068807), which is the region of the input space a given feature is looking at. Given the size of our structures of interest and our network architecture, we use 256 x 256 pixel tiles. Also, training on smaller tiles allow us to use bigger batch size so the training converges faster.

We have additional options like resizing, flatfield correction, and creating masks. The data we're working with is already background corrected so we can safely skip that part. We also don't need to resize the images. We would however like to create masks based on our target data. The reason for that is that we would like to avoid training on background only, that is empty or near empty tiles. A threshold we found reasonable is to make sure that 25% or more pixels of our tiles contain signal.



#### masks

We would like to generate binary masks based on our target channel. In microDL we have two types of global binary thresholding methods builtin, [Otsu](https://en.wikipedia.org/wiki/Otsu%27s_method) and [unimodal](https://users.cs.cf.ac.uk/Paul.Rosin/resources/papers/unimodal2.pdf) (or Rosin) thresholding.
Let's load an image, generate masks and plot them side by side.

In [ ]:
# Load image
position = 5
im_path = os.path.join(INPUT_DIR, f'img_405_t000_p{position:03}_z010.tif')
im = im_utils.read_image(im_path)
# Clip top and bottom 1% of histogram for better visualization
im = norm_utils.hist_clipping(im, 1, 99)

mask_otsu = mask_utils.create_otsu_mask(im)
mask_rosin = mask_utils.create_unimodal_mask(im)

fig, ax = plt.subplots(1, 3)
fig.set_size_inches(20, 15)
ax = ax.flatten()
ax[0].imshow(im, cmap='gray')
ax[0].set_title('Fluorescence',fontsize=20)
ax[1].imshow(mask_otsu, cmap='gray')
ax[1].set_title('Otsu thresholding',fontsize=20)
ax[2].imshow(mask_rosin, cmap='gray')
ax[2].set_title('unimodal thresholding',fontsize=20)
for a in ax: a.axis('off')

plt.show()
plt.close('all')

#### configuration

For analyzing large amounts of data and to keep track of the parameters, we run microDL through command line interfaces (CLIs) using configuration files. For each module we specify the path to a config file as a command line argument. Since we're using Jupyter Notebook for this tutorial we will instead load the preprocessing config and update it. Look for `...` to fill in the blanks

In [ ]:

config_path = os.path.join(
    module_path,
    'config_files/config_preprocess.yml',
)
preproc_config = aux_utils.read_config(config_path)

# We're not doing resizing so let's remove that key
if 'resize' in preproc_config:
    preproc_config.pop('resize')
# We're not doing flatfield correction either
if 'flat_field' in preproc_config:
    preproc_config.pop('flat_field')
    
# We need to change INPUT_DIR to point to where our image data is located
preproc_config['input_dir'] = INPUT_DIR

# And where we want to store our preprocessed data
preproc_config['output_dir'] = TILE_DIR

# Set which channels we want to preprocess and if they should be normalized
preproc_config['channel_ids'] = [0, 3]
preproc_config['normalize']['normalize_channels'] = [True, True]
preproc_config['tile']['depths'] = [1, 1]

# Set the channels used for generating masks
preproc_config['masks']['channels'] = 0

# Set the number of workers to the number of available cores
preproc_config['num_workers'] = 8

<div class="alert alert-block alert-info"><h4>
    TODO 2: Choose a masking strategy for this exercise that you think will result in the optimal balance between foreground and background pixels. This begs a question: for regression problems, which pixels are foreground?

As you can see, the Otsu threshold captures only the very bright structures and misses dim structures. Rosin thresholding does a better job picking up these dim structures.

In [ ]:
preproc_config['masks']['mask_type'] = ... # TODO 2: your choice of thresholding algorithm here.
preproc_config['pos_ids'] = [3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 40, 41, 42]
pp.pprint(preproc_config)


#### generate patches/tiles
Now it's time to run the preprocessing: the runtime should be around 5 minute. 
***
Notes:
1. If you get an error about "Don't specify a mask_dir", try reloading the config by running the last two blocks.
2. If above step takes too long due to network or data i/o bottlenecks, you can copy the patches from this location to the TILE_DIR

In [ ]:
preproc_config, runtime = preprocess.pre_process(preproc_config);
print("Preprocessing took {} seconds".format(runtime))
# Save the final config and run time
preprocess.save_config(preproc_config, runtime)

You can navigate to your output_dir and take a look at what was generated. You will find a mask_dir containing masks, a tile_dir containing tiles and JSON file containing the all the information that was used to generated the preprocessed data. Let's take a look at a few tiles. 

#### visualize 10 random patches

In [ ]:
tile_dir = preproc_config['tile']['tile_dir']
# tile_dir =os.path.join(os.path.expanduser('~'), '04_image_translation_data/tmp/tile_small/tiles_256-256_step_128-128' )
print(tile_dir)
frames_meta = pd.read_csv(os.path.join(tile_dir, 'frames_meta.csv'))
dna_tiles = frames_meta[frames_meta.file_name.str.match('im_c000_*')]
subset = np.random.choice(dna_tiles.shape[0], 10, replace=False)

fig, ax = plt.subplots(3, 10)
fig.set_size_inches(20, 10)
# ax = ax.flatten()
for i, row  in enumerate(subset):
    dna_tile_path = os.path.join(tile_dir, frames_meta.iloc[row].file_name)
    phase_tile_path = dna_tile_path.replace('c000','c003')
    mask_tile_path = dna_tile_path.replace('c000','c004')
    
    dna = im_utils.read_image(dna_tile_path)
    phase = im_utils.read_image(phase_tile_path)
    mask = im_utils.read_image(mask_tile_path)
    ax[0,i].imshow(np.squeeze(dna), cmap='gray'); ax[0,i].axis('off')
    ax[1,i].imshow(np.squeeze(phase), cmap='gray'); ax[1,i].axis('off')
    ax[2,i].imshow(np.squeeze(mask), cmap='gray'); ax[2,i].axis('off')
  
plt.show()
plt.close('all')


### Checkpoint 2

<div class="alert alert-block alert-success"><h4>
    
Let us know by slack if you have successfully generated tiles for image translation.

## 3. 2D virtual staining (slice$\rightarrow$slice)

### Configure training
Now that we've preprocessed our data we're ready to train. During this exercise, we will use phase or retardance image to predict two different fluorescent channels, actin and nuclei.
In our dataset, the channel names and indices are the following:

* Retardance: channel name Retardance, index 2
* Phase: channel name phase, index 3
* Actin:channel name 568, index 1
* Nuclei: channel name 405, index 0

We specify the network architecture and training parameters using another config file. Let's load a base 2D training config file and take a look.



In [ ]:
config_path = os.path.join(
    module_path,
    'config_files/config_train.yml',
)
train_config = aux_utils.read_config(config_path)
# Set the data directory to the directory we generated during preprocessing
train_config['dataset']['data_dir'] = os.path.join(TILE_DIR, 'tiles_256-256_step_128-128')

# We also need to specify where we want to store our model and all related data generated by training
# This directory will be created if it doesn't already exist
train_config['trainer']['model_dir'] = MODEL_DIR

# Predict dna (channel 1) from phase (channel 3)
train_config['dataset']['input_channels'] = [3]
train_config['dataset']['target_channels'] = [0]


### Train the model!

There are many parameters of a model architecture and training process that a DL practitioner optimizes by hand - these are broadly termed hyper-paramters. They are distinct from parameters of the layers of the neural network learned from the data. Following are key hyper-parameters for virtual staining and good defaults.

#### Final activation layer
Virtual staining is a regression task,and therefore the final activation layer needs to be linear. For a binary segmentation, we use a sigmoid to move output towards either zero or one. For regression, we would like the prediction to match the dynamic range of the target.

##### Loss
Common choices for regression are the mean squared error (MSE) and the mean absolute error (MAE) between the target image y and the estimated image y':
\begin{equation*}
MSE = \sum_{p} (y_p - y_p')^2,
\end{equation*}

\begin{equation*}
MAE = \sum_{p} |y_p - y_p'|
\end{equation*}
for each pixel index p.

You can try both by changing `train_config['trainer']['loss']`. The names in microDL for losses are mse_loss and mae_loss, respectively. You can also try other custom losses by using the loss function names [here](https://github.com/czbiohub/microDL/blob/master/micro_dl/train/losses.py), or any standard [Keras loss function](https://keras.io/losses/) by specifying the loss function name defined by Keras. 

##### Optimizer
Adam is a good default optimizer. You can read more about different deep learning optimizers [here](http://ruder.io/optimizing-gradient-descent/), and you can change the optimizer you'd like to use in your training by changing the variable `train_config['trainer']['optimizer']['name']` to any of the Keras optimizers listed [here](https://keras.io/optimizers/).

##### Learning Rate
If the learning rate is too small your training might take a very long time to converge, and if it's too big it might not converge at all. It's worth trying some different values and see what happens with convergence.

##### Dropout
Since we're working with a very small dataset in exploratory training, chances are that your network will overfit to your training data. It's worth exploring `train_config['network']['dropout']` and to see if increasing those variables can reduce overfitting.

##### Number of filters
The number of filters in each layer of the model controls the model capacity. This parameter is `train_config['network']['num_filters_per_block']`. Too large model capacity can lead to overfitting and not necesssarily better results.

##### Augmentation
The data is flipped and rotated randomly to diversify the training set and mitigate overfitting.

##### Other?
If you have extra time or are curious about the other variables, feel free to explore.

<div class="alert alert-block alert-info"><h4>
    TODO 3: Pick a loss function and other hyper-parameters you think are right for the regression problem. 

In [ ]:
# Enable data augmentation.
train_config['dataset']['augmentation'] = True

train_config['trainer']['loss'] = ... # TODO 3: your choice of loss function here.
# Set maximum number of epochs to 7 so we can explore difference parameters quickly
train_config['trainer']['max_epochs'] = 7
# Batch size of 64 just about occupies 16GB RAM.
train_config['trainer']['batch_size'] = 64

# Set number of filters per block. The size of this vector sets the depth of U-Net.
train_config['network']['num_filters_per_block'] = [16, 32, 48, 64, 80]
# You can also update other training hyper-parameters here.
pp.pprint(train_config)


In [ ]:
# First, specify the gpu that you want to run training on, or "None" to select the gpu with most memory available
gpu_id = 0
gpu_id, gpu_mem_frac = train_utils.select_gpu(gpu_ids=gpu_id)

Try restarting the notebook kernel if the gpu memory is occupied and you run into errors about "can't creat training session".
Training 7 epochs should take no more than 10 minutes if you're on a GPU.

In [ ]:
train.run_action(
    action='train',
    config=train_config,
    gpu_ids=gpu_id,
    gpu_mem_frac=gpu_mem_frac,
)


We've completed our first training. Let's take a look at what happened during training by opening a history log file.

In [ ]:
history = pd.read_csv(os.path.join(MODEL_DIR, 'history.csv'))
history

In [ ]:
# Plot training vs. validation loss
fig, ax = plt.subplots()
fig.set_size_inches(12, 9)
ax.plot(history['epoch'], history['loss'], 'r')
ax.plot(history['epoch'], history['val_loss'], 'b')
ax.set_xlabel('Epoch')
ax.set_ylabel('Loss')
plt.legend(['train', 'validation'])
plt.show()

### Checkpoint 3

<div class="alert alert-block alert-success"><h4>
Please let us know by slack whether your training converged.
    
**Does it look like the model is overfitting? How can you tell?**



## 4. Predictions on test set <a class="anchor" id="predict"></a>

We'd also like to see how well the model performs predictions. For that we will have to run inference on our test dataset. We will run model inference on the full size 2048 X 2048 image instead of on tiles in training. 

<div class="alert alert-block alert-info"><h4>

TODO 4: Why can we run the model inference with input size different from when used during training? And what are the benefits of doing that?  

... TODO 4: your response

### configuration

In [ ]:
config_path = os.path.join(
    module_path,
    'config_files/config_inference.yml',
)
inf_config = aux_utils.read_config(config_path)
inf_config['image_dir'] = INPUT_DIR 
inf_config['preprocess_dir'] = TILE_DIR
inf_config['model_dir'] = MODEL_DIR
inf_config['dataset']['input_channels'] = [3]
inf_config['dataset']['target_channels'] = [0]
inf_config['metrics']['metrics_orientations'] = ['xy']
pp.pprint(inf_config)

In [ ]:
split = aux_utils.read_config(os.path.join(MODEL_DIR , 'split_samples.json'))
split

### Run prediction

In [ ]:
inference_inst = image_inf.ImagePredictor(
                train_config=train_config,
                inference_config=inf_config,
                preprocess_config=preproc_config,
                gpu_id=gpu_id,
                gpu_mem_frac=gpu_mem_frac,
            )
inference_inst.run_prediction()

There should be a new subdirectory created in the model directory with the predictions and the metrics. Use glob to see what files were generated during inference.

Plot an example of input, target and prediction side by side.

In [ ]:
pred_dir = os.path.join(MODEL_DIR, 'predictions')
position = split['test'][0]
z = 12
fig, ax = plt.subplots(1, 3)
fig.set_size_inches(20, 15)
ax = ax.flatten()
for a in ax: a.axis('off')

im_path = os.path.join(INPUT_DIR, f'img_phase_t000_p{position:03}_z{z:03}.tif')
im = im_utils.read_image(im_path)
im = norm_utils.hist_clipping(im, 1, 99)
ax[0].imshow(im, cmap='gray'); ax[0].set_title('Input: Phase', fontsize=20)
im_path = os.path.join(INPUT_DIR, f'img_405_t000_p{position:03}_z{z:03}.tif')
im = im_utils.read_image(im_path)
im = norm_utils.hist_clipping(im, 1, 99)
ax[1].imshow(im, cmap='gray'); ax[1].set_title('Target: DNA', fontsize=20)
im_path = os.path.join(pred_dir, f'img_405_t000_p{position:03}_z{z:03}.tif')
im = im_utils.read_image(im_path)
im = norm_utils.hist_clipping(im, 1, 99)
ax[2].imshow(im, cmap='gray'); ax[2].set_title('Prediction of DNA', fontsize=20)
plt.show()
plt.close('all')

The prediction is blurrier than the target. One reason you can't exactly mimic the nuclei image is that the input channel carries partial information about the structure and the random noise cannot be learned.

Another reason for blurry prediction is that loss functions like MSE or MAE tend to generate blurrier prediction because these loss functions generate an "average" prediction when there are multiple possible predictions.  

Also, we've here used a very limited amount of data. To get higher correlation we will need to include much more data and run training overnight.

Speaking of correlation, let's open the inference meta file and inspect the metrics comparing predictions and targets.

Each metric reports on different aspects of the fit between predictions (P) and target (T).  
* $R^2$ is the coefficient of determination = $1- \frac{MSE(P,T)}{\sigma^2(T)}$. It is sensitive to difference in the mean values.
* The Pearson correlation coefficient is
$$ r(T,P) = \frac{\sigma_{TP}}{\sigma_T \sigma_P} $$ It is insensitive to the mean value, and reports if two variables vary linearly.  
* SSIM is a multi-component metric [developed to represent human vision](http://www.cns.nyu.edu/pub/lcv/wang03-reprint.pdf). It is a single metric that reports on similarity of luminance (local mean), contrast (local variance), and structure (local pearson correlation) between two images.


In [ ]:
metrics_meta = pd.read_csv(os.path.join(pred_dir, 'metrics_xy.csv'))
metrics_meta.head()

Get the median correlation of all rows in the inference csv file.

In [ ]:
print("Median Pearson correlation: {:.2f}".format(metrics_meta['corr'].median()))
print("Median SSIM: {:.2f}".format(metrics_meta['ssim'].median()))

### Checkpoint 4
<div class="alert alert-block alert-success"><h4>

**Post the median metrics achieved with your model on slack along with key choices of hyper-parameters.** 

**Which aspects of data and the model lead to  less-than-perfect match?** 

## 5. Model tuning <a class="anchor" id="tuning"></a>

Now it's time for you to experiment. You can try modeling a different channel (use retardance channel to predict F-actin) or play around with different settings in the train_config and rerun the training. What do you think will help improve the model?

## Bonus 1: Training with different input-output pairs.

With preprocessed tiles, you are setup to explore how accurately you can predict one channel from the other. Some questions you may want to explore:

* How accurately can you predict the phase images from images of nuclei? Is the accuracy of prediction symmetric?

* Which label-free channel (phase or retardance) provides more accurate prediction of F-actin?

##  Bonus 2: Training on larger dataset (slice$\rightarrow$slice) overnight.

Hopefully above exploration has led you to set of parameters to predict nuclei with decent accuracy. You can now set up a model to train on larger dataset (30 FOVs) and evaluate if model accuracy improves as a result when you comeback. 

We will need change the position ids in the pre-process config to have 30 FOVs and re-run preprocessing. The position ids of 30 FOVs are:

[3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]

You can pick training/model parameters based on your parameter exploration from Session 1. A few tips for picking training parameters for overnight training:
- **Make sure you write preprocessed data and model trained on this large set to new folders.**
- Increase maximum number of epochs and early stopping patience to at least 200 and 10 so the training will run longer
- Increase the number of filters in the model to increase the model capacity. You might need to use smaller batch size so the model can fit into the GPU memory You might want to add some dropout as well to avoid overfitting if you increase the number of filters 
- Use lower learning rate. We used higher learning rate to make training converge faster
- Compare the mean and standard deviation of test metrics.

